## Troll and Toad Card Price Scraper

In [1]:
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup as bs

In [2]:
test_data = {"Card_Name": ["Aegislash", "Pikachu and Zekrom GX", "Darkrai VSTAR"],
             "Card_Number": ["177/185", "SM248", "GG50/GG70"]}
test_df = pd.DataFrame(test_data)
test_df

,Card_Name,Card_Number
0,Aegislash,177/185
1,Pikachu and Zekrom GX,SM248
2,Darkrai VSTAR,GG50/GG70


In [3]:
exempt_cards = ["pokemon-oversized-cards", "prize-pack-series"]
search_urls = []
price_urls = []
card_prices = []

In [4]:
for ind in test_df.index:
    card_query = ""
    card_words = test_df.Card_Name[ind].split()
    for word in card_words:
        card_query += "{}+".format(word)
    if "/" in test_df.Card_Number[ind]:
        card_number, set_number = test_df.Card_Number[ind].split("/")
        card_query += "{}%2F{}".format(card_number, set_number)
    else:
        card_query += "{}".format(test_df.Card_Number[ind])
    search_url = "https://www.trollandtoad.com/category.php?selected-cat=0&search-words={}".format(card_query)
    search_urls.append(search_url)

In [5]:
for url in search_urls:
    search_page = requests.get(url)
    search_parsed_page = bs(search_page.text, "html.parser")
    card_options = search_parsed_page.find_all("a",  class_ = "card-text")
    for option in card_options:
        is_exempt = False
        option_string = option["href"].replace("/", " ")
        for exempt in exempt_cards:
            if exempt in option_string:
                is_exempt = True
        if is_exempt:
            continue
        else:
            price_url = "https://www.trollandtoad.com{}".format(option["href"])
            price_urls.append(price_url)
            break

In [6]:
for url in price_urls:
    price_page = requests.get(url)
    price_parsed_page = bs(price_page.text, "html.parser")
    card_prices.append(price_parsed_page.find(id = "sale-price").text)

In [7]:
card_prices = list(map(float, card_prices))
depreciation = [x * 0.3 for x in card_prices]
card_prices = np.subtract(card_prices, depreciation)
test_df["Estimated_Card_Price"] = card_prices
test_df

,Card_Name,Card_Number,Estimated_Card_Price
0,Aegislash,177/185,1.953
1,Pikachu and Zekrom GX,SM248,4.543
2,Darkrai VSTAR,GG50/GG70,12.593
